In [1]:
# Define function that pulls coverage data from FCC API (https://geo.fcc.gov/api/contours/#!/coverage/get_coverage_format)

import json
import pandas as pd

## Key:
# lat = Latitude
# lon = Longitude
# nradial = Number of radials
# rcamsl = Radiation Center Above Mean Sea Level (meters)
# field = field strength (dBu)
# erp = Effective Radiation Power (kW)
# curve = Type of propagation curve, range is 0-2. 0: use F(50, 50) curve. 1: use F(50, 10) curve. 2: use F(50, 90) curve

import requests

def get_coverage_data(lat, lon, rcamsl, erp, nradial = 36, field = 60, curve = 0, unit = 'm', pop = 'false', area = 'false'):
    results = {}
    # Define the base URL
    base_url = "https://geo.fcc.gov/api/contours/coverage.json"
    
    # Define parameters
    params = {
        "serviceType": "fm",
        "lat": lat,
        "lon": lon,
        "nradial": nradial,
        "rcamsl": rcamsl,
        "field": field,
        "erp": erp,
        "curve": curve,
        "pop": pop,
        "area": area,
        "unit": unit
    }
    
    # Make the request
    response = requests.get(base_url, params=params)
    
    # Check for successful response
    if response.status_code == 200:
        # Pull JSON data
        data = response.json()
        
        # Pull antenna coord
        antenna_lat = data['features'][0]['properties']['antenna_lat']
        antenna_lon = data['features'][0]['properties']['antenna_lon']

        # Store
        results['lat'] = antenna_lat
        results['lon'] = antenna_lon

        # Pull vertices
        results['vertices'] = data['features'][0]['geometry']['coordinates'][0][0]
        return results
    else:
        print("Error:", response.status_code)
        return None



In [2]:
# # test
# results_test = get_coverage_data(lat = 42.3616942334, lon = -71.0855243438, rcamsl = 110, erp = 0.64)

# import folium

# # Define center of map
# m = folium.Map(location=(results_test['lat'],results_test['lon']), zoom_start=12, width='50%', height='40%')

# # Add contour as a polyline
# folium.PolyLine(locations=[[y, x] for x, y in results_test['vertices']], color="blue").add_to(m)
# m

In [3]:
data = pd.read_csv('Exports/Data/station_eng_data.csv')
data.head()

,eng_call_sign,eng_city,eng_erp,eng_frequency,eng_latitude,eng_longitude,eng_rcamsl,eng_state,stat_biamarketid,stat_biastationcode,...,stat_initial_hh_exp,stat_letters,stat_market,stat_market_name,stat_market_state,stat_rank,stat_rating,stat_startdate,stat_supermarket,match_reason
0,WVVE,Stonington,3.000,102.3,41.406389,-71.837500,137.0,CT,262.0,17132.0,...,NaN,WAXK,New London CT,New London,['CT'],171.0,4.15,1981.0,NaN,one_to_one
1,K272BR,Jefferson City,0.078,102.3,38.636944,-92.213333,279.0,MO,196.0,20865.0,...,NaN,KBXR,Columbia MO,Columbia,['MO'],249.0,5.00,1994.0,NaN,one_to_one
2,WJSK,Lumberton,3.000,102.3,34.599444,-79.009167,122.0,NC,136.0,15938.0,...,NaN,WFNC,Fayetteville NC,Fayetteville,['NC'],129.0,0.50,1964.0,NaN,one_to_one
3,KGGY,Dubuque,1.650,102.3,42.541111,-90.612778,374.0,IA,200.0,11903.0,...,NaN,KXGE,Dubuque IA,Dubuque,['IA'],225.0,5.15,1980.0,NaN,100.0
4,WGL,Auburn,3.000,102.3,41.333611,-85.052222,355.0,IN,108.0,15601.0,...,1999.0,WCKZ,Ft. Wayne IN,Ft. Wayne,['IN'],103.0,0.20,1967.0,NaN,one_to_one


In [4]:
data_input = data[['eng_erp', 'eng_rcamsl', 'eng_latitude', 'eng_longitude']].copy()
assert data_input.isna().sum().sum() == 0
data_input.head()

,eng_erp,eng_rcamsl,eng_latitude,eng_longitude
0,3.000,137.0,41.406389,-71.837500
1,0.078,279.0,38.636944,-92.213333
2,3.000,122.0,34.599444,-79.009167
3,1.650,374.0,42.541111,-90.612778
4,3.000,355.0,41.333611,-85.052222


In [5]:
# How long will this take?
import time
wait_time = 11

seconds = wait_time*len(data)
minutes = seconds/60
hours = minutes/60
print(f"Hours: {hours: .2f}")

Hours:  15.00


In [ ]:
import json

try:
    with open('Exports/Data/4.coverage_coords.json', 'r') as f:
        results = json.load(f)
except FileNotFoundError:
    results = {}

results = {int(k): v for k, v in results.items()}

In [7]:
print(f'results: {results}')

indices_processed = pd.Series(results.keys())

print(f'min: {indices_processed.min()}')
print(f'max: {indices_processed.max()}')
missing_indices = set(range(indices_processed.min(), indices_processed.max() + 1)) - set(indices_processed)
if missing_indices:
    print(f"Missing indices: {sorted(missing_indices)}")
else:
    print("All indices are processed.")

results: {0: {'lat': 41.4063888889, 'lon': -71.8375, 'vertices': [[-71.8375, 41.5684230249], [-71.7959082232, 41.582818271], [-71.758703307, 41.5683340609], [-71.7148861768, 41.5652220052], [-71.6639083332, 41.5610553652], [-71.6221857068, 41.5414157853], [-71.5830665294, 41.5161009875], [-71.5501004675, 41.4843947378], [-71.5356705226, 41.4458877893], [-71.536564876, 41.4059968412], [-71.5327926198, 41.3656635354], [-71.5447468, 41.3259998699], [-71.5662756242, 41.2884077523], [-71.5918137569, 41.2511342205], [-71.6305944193, 41.2207310715], [-71.677169727, 41.1973227573], [-71.7279026273, 41.1797016044], [-71.781087003, 41.1655245168], [-71.8375, 41.1591149068], [-71.8936357757, 41.1667125773], [-71.9484405386, 41.1769130197], [-72.0011102315, 41.1930294525], [-72.0457503408, 41.2195197332], [-72.0802153007, 41.2530191735], [-72.1056668648, 41.2897436895], [-72.12080849, 41.3286080463], [-72.113254142, 41.3695700154], [-72.1116925759, 41.406063424], [-72.0865237337, 41.4390377535], [

In [ ]:
# Running API
for i, row in data_input.iterrows():
    if i in results:
        # Skip rows that are already processed
        print(f'Skipping already processed row {i}')
        continue

    if i % 50 == 0 and i != 0:
        print("Checkpoint save")
        print(f"Processed rows: {list(results.keys())[:5]}... (total {len(results)} rows)")

        with open('Exports/Data/4.coverage_coords.json', 'w') as f:
            json.dump(results, f)
    
    print(f'Processing row {i}')
    print(f'Completed: {len(results.keys()) / len(data_input):.2%}')

    lat = row['eng_latitude']
    lon = row['eng_longitude']
    rcamsl = row['eng_rcamsl']
    erp = row['eng_erp']
    
    try:
        result = get_coverage_data(lat=lat, lon=lon, rcamsl=rcamsl, erp=erp)
        results[i] = result
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue

    time.sleep(wait_time)

# Final save of results
print("Final save of results")
with open('Exports/Data/4.coverage_coords.json', 'w') as f:
    json.dump(results, f)

Skipping already processed row 0
Skipping already processed row 1
Skipping already processed row 2
Skipping already processed row 3
Skipping already processed row 4
Skipping already processed row 5
Skipping already processed row 6
Skipping already processed row 7
Skipping already processed row 8
Skipping already processed row 9
Skipping already processed row 10
Skipping already processed row 11
Skipping already processed row 12
Skipping already processed row 13
Skipping already processed row 14
Skipping already processed row 15
Skipping already processed row 16
Skipping already processed row 17
Skipping already processed row 18
Skipping already processed row 19
Skipping already processed row 20
Skipping already processed row 21
Skipping already processed row 22
Skipping already processed row 23
Skipping already processed row 24
Skipping already processed row 25
Skipping already processed row 26
Skipping already processed row 27
Skipping already processed row 28
Skipping already process